# Рекурентные сети для обработки последовательностей

Вспомним все, что мы уже знаем про обработку текстов:
- Компьютер не понимает текст, поэтому нам нужно его как-то закодировать - представить в виде вектора
- В тексте много повторяющихся слов/лишний слов - нужно сделать препроцессинг:
    - удалить знаки препинания
    - удалить стоп-слова
    - привести слова к начальной форме (**стемминг** и **лемматизация**)
    
    
- После этого мы можем представить наш текст (набор слов) в виде вектора, например, стандартными способами:
    - **CounterEncoding** - вектор длины размер нашего словаря
        - есть словарь vocab, который можем включать слова, ngram-ы
        - каждому документу $doc$ ставим в соответствие вектор $vec\ :\ vec[i]=1,\ если\ vocab[i]\ \in\ doc$
    - **TfIdfVectorizer** - вектор длины размер нашего словаря
        - есть словарь vocab, который можем включать слова, ngram-ы
        - каждому документу $doc$ ставим в соответствие вектор $vec\ :\ vec[i]=tf(vocab[i])*idf(vocab[i]),\ если\ vocab[i]\ \in\ doc$
    
        $$ tf(t,\ d)\ =\ \frac{n_t}{\sum_kn_k} $$
        $$ idf(t,\ D)\ =\ \log\frac{|D|}{|\{d_i\ \in\ D|t\ \in\ D\}|} $$
        
, где 
- $n_t$ - число вхождений слова $t$ в документ, а в знаменателе — общее число слов в данном документе
- $|D|$ — число документов в коллекции;
- $|\{d_i\ \in\ D\mid\ t\in d_i\}|$— число документов из коллекции $D$, в которых встречается $t$ (когда $n_t\ \neq\ 0$).



Это база и она работает. Мы изучили более продвинутые подходы: эмбединги и сверточные сети по эмбедингам. Но тут есть проблема: любой текст - это последовательность, ни эмбединги, ни сверточные сети не работают с ним как с последовательностью. Так давайте попробуем придумать архитектуру, которая будет работать с текстом как с последовательностью, двигаясь по эмбедингам и как-то меняя их значения.

# 1. RNN

Рассмотрим архитектуру, которая позволяет решать задачи свзанные с последовательностями. Будем изучать по мотивам известной [статьи](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) от [Андрея Карпатова](http://karpathy.github.io/).

<img src='https://drive.google.com/uc?export=view&id=1hYDawc813JLg6vHdvVlbZH-S7ADtwGX6' width=1000>

1) На первой картинке представлен пример нашей обычной нейронной сети. Она что-то получает на вход, что-то с ним делает и выдает выходные значения. Все фиксированных размеров.  
2) Мы познакомимся с архитектурой, которая может на вход получить один вход, а на выходе иметь несколько.   
3) И наоборот, на вход подать несколько (причем неизвестного количества), а на выходе одно значение. Один из примеров такой задачи - это [sentimental analysis](https://monkeylearn.com/sentiment-analysis/). Т.е. на вход идет некое предложение, а на выход мы хотим получить одно значение. Например, оно позитивное или негативное или еще какое.  
4) Или же могут быть варианты многие ко многим. На вход подаем некоторое переменное количество параметров, а на выходе другое тоже переменое количество параметров.  Пример такой задачи - это перевод текста (перевод на другие иностранные языки). Или другой вариант многие ко многим - это когда количество параметров на входе фиксированно и соотвествует выходу.

Собственно, как выглядит архитектура, которая позволяет работать с таким переменным количетсвом параметров на вход и на выход?

## 1.1 Основная Идея
Возьмем обычную нейронную сеть. 

<img src='https://drive.google.com/uc?export=view&id=1ccQJE-E40zNb8U1pfalvxdMXqabepwZp' width=300>

У нас есть вход, выход и т.д. Посередине один линейный слой. И он выдает какой-то сигнал на выходе.  
И мы возьмем и начнем этот же сигнал (этот же вектор) подавать на вход сети самой себе на следующем шаге:


<img src='https://drive.google.com/uc?export=view&id=1zPx_5hIX8_1Q0EPEUWjcOn7hNGHgALWi' width=600>

У нас есть какая-то последовательность данных. Например назовем их a, b, c. Для первого элемента последовательности мы просто прогнали сеть. Чтобы вход скрытого слоя всегда получал одно и тоже значение, мы сначала даем ему просто какой-то нулевой вектор и то что выдал предыдущий слой. А для следующей сети из последовательности мы возьмем то, что выдала сеть на прошлом шаге и передадим ей. Т.е. у такой сети есть возможность передать самой себе в будущем некоторое состояние. Ну и продолжим так делать. 

<img src='https://drive.google.com/uc?export=view&id=1pHueSBIHuRxh1DJTfkQny007g3dh_pCw' width=800>

Мы тренируем такую систему, где промежуточный выход с сети прошлого шага дают на вход сети на следующем шаге. Это значит, что она может выучить некоторое  представление входа. Как-то его закодировать в вектор и передать самой себе на следующем шаге, чтобы знать, что происходило раньше. И таким образом результирующий выход сети, он уже зависит от всей накопленной последовательности. И это дает нам возможность на основе всей последовательности выдать выход.  
И тот факт, что мы даем на вход сети самой себе - это некоторая рекурсия и поэтому такая архитектура называется так, как она называется.

## 1.2 Давайте посмотрим как это примерно выглядит в коде:

```

class RNN:
  # ...
  def step(self, x):
    # update the hidden state
    self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))
    # compute the output vector
    y = np.dot(self.W_hy, self.h)
    return y

```



Вот наш шаг - т.е. forward, прямой проход. Он выглядит следующим образом:  
Мы делаем матричное произведение весов W и текущего состояния h и добавляем тоже произведение весов W на вход x. и получаем новый h.

Давайте о том же самом, только на другой картинке:

<img src='https://drive.google.com/uc?export=view&id=1cJVaUAJZdjaFw65zR_2QyJ9nEgmMV8td' width=800>

У нас есть набор X, где t - элементы последовательности. Мы сливаем h и X_t  вместе, прогоняем через слой, через тангенсальную функцию активации. и получаем следующие h.

Почему tanh? А например не ReLU? Второй всегда отрезает отрицательную часть сигнала. И это имеет такое последствие, что сигнал, который проходит через сеть никогда не сможет стать отрицательным. И это значит, что когда через много слоев проходим - то сигнал может только расти. Поэтому пользуются именно tanh, что бы были возможности получить и плюс, и минус.

## 1.3 Количество слоев

Можно так же настэкать большое количество слоев, как и везде.   
Вот так выглядит рекурентная сеть с несколькими слоями:

<img src='https://drive.google.com/uc?export=view&id=175Ij-M6mGkW7zo2n-_u9fM-cPezPcTo8' width=800>

То есть обычным образом можно прогнать сигнал через слои сети. И при этом каждый из слоев выдает своему эквивалентному слою в следующей итерации сети некоторое скрытое состояние. Чем больше слоев, тем сеть обладает большей обобщающей способностью. И каждый слой на своем уровне понимания может себе в будущем передать состояние. При этом веса у каждого эквивалентного слоя одни и те же.

## 1.4 Пример

Вот у нас есть задача генерации текста. Разберем наш текст на последовательность символов.  

[BOS], "h", "e", "l", "l", "o"

У нас будет специальный вспомогательный символ BOS - begin of sentense.  

И мы будем тренировать сеть, такую, что по каждому элементу будем предсказывать следующий. Т.е.

[BOS] -> 'h'  
'h' -> 'e'  
'e' -> 'l'  
'l' -> 'l'  
'l' -> 'o'  
'o' -> [EOS]  

Вот пример как это может выглядеть:


<img src='https://drive.google.com/uc?export=view&id=1h6ABYQwTIxCAKdxjgRiIjkf_Go1qvojW' width=800>

Элементы нашей последовательности мы можем представить как one-hot представление. Оно все проходит и на выходе предсказывает следующий символ в one-hot представлении. И если это дело долго тренировать, то на выходе такая сеть начинает генерировать реально хороший текст.

In [ ]:
!pip install stop-words pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 8.2 MB 39.3 MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=000f288bb4e7da0dd39177cc6d4e60828446f1241a5775130f7e67243381a942
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=1332ffac7d97139ae436dd3d639db1e65078405c4d91f6678a95e9913602fae4
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built stop-words docopt


In [ ]:
# попробуем запрограммировать простую рекурентную сеть. Возьмем датасет с прошлого занятия

import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

Ссылка на google drive: https://drive.google.com/file/d/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901' -O data.zip

--2022-08-22 15:40:10--  https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901
Resolving drive.google.com (drive.google.com)... 142.251.12.113, 142.251.12.138, 142.251.12.139, ...
Connecting to drive.google.com (drive.google.com)|142.251.12.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uednrkbn9fsu2h997psdhj069h3o39to/1661182800000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=2e32c92d-b832-4b4f-9060-d9c3c0294ea4 [following]
--2022-08-22 15:40:15--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uednrkbn9fsu2h997psdhj069h3o39to/1661182800000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=2e32c92d-b832-4b4f-9060-d9c3c0294ea4
Resolving doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 74.125.24.132, 2404:6

In [ ]:
!unzip data.zip

Archive:  data.zip
  inflating: train.csv               
  inflating: val.csv                 


In [ ]:
import pandas as pd

df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

In [ ]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [ ]:
sw = set(get_stop_words("ru"))
puncts = set(punctuation)
morpher = MorphAnalyzer()


def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
    txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [ ]:
from tqdm import tqdm 
tqdm.pandas()

df_train['text'] = df_train['text'].progress_apply(preprocess_text)
df_val['text'] = df_val['text'].progress_apply(preprocess_text)

100%|██████████| 22683/22683 [00:27<00:00, 829.65it/s]


In [ ]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [ ]:
max_words = 2000
max_len = 20
num_classes = 1

# Training
epochs = 5
batch_size = 512
print_batch_n = 100

In [ ]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [ ]:
from nltk.probability import FreqDist

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [ ]:
tokens_filtered_top[:10]

['rt',
 'd',
 'хотеть',
 'новый',
 'завтра',
 'мой',
 'вообще',
 'хороший',
 'любить',
 'немочь']

In [ ]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
# vocabulary

In [ ]:
import numpy as np


def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

In [ ]:
%%time
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

CPU times: user 19.9 s, sys: 111 ms, total: 20 s
Wall time: 20 s


In [ ]:
x_train.shape

(181467, 20)

In [ ]:
x_train[1]

array([  1, 196, 429, 161,   5,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch


class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [ ]:
train_dataset = DataWrapper(x_train, df_train['class'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DataWrapper(x_val, df_val['class'].values)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


<img src='https://drive.google.com/uc?export=view&id=175Ij-M6mGkW7zo2n-_u9fM-cPezPcTo8' width=800>

In [ ]:
from torch import nn


class RNNFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers=2, batch_first=True, )
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)

        rnn_out, ht = self.rnn(x) 
        # rnn_out: тензор с выходными фичами с последнего слоя для каждого t
        # h_t: тензор с последними скрытыми состояниями по слоям

        if self.use_last:
            last_tensor = rnn_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(rnn_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        
        return torch.sigmoid(out)

In [ ]:
rnn_init = RNNFixedLen(max_words, 128, 20, use_last=False)
optimizer = torch.optim.Adam(rnn_init.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [ ]:
print(rnn_init)
print("Parameters:", sum([param.nelement() for param in rnn_init.parameters()]))

RNNFixedLen(
  (embeddings): Embedding(2000, 128, padding_idx=0)
  (rnn): RNN(128, 20, num_layers=2, batch_first=True)
  (linear): Linear(in_features=20, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 259861


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
rnn_init.to(device)
device

'cuda'

In [ ]:
rnn_init = rnn_init.to(device)
rnn_init.train()
th = 0.5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    rnn_init.train()
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)

        # обнуляем градиент
        optimizer.zero_grad()
        outputs = rnn_init(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
        
    # выводим статистику о процессе обучения
    rnn_init.eval()
    
    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
          f'Step [{i + 1}/{len(train_loader)}]. ' \
          f'Loss: {loss:.3f}. ' \
          f'Acc: {running_right / running_items:.3f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

    # выводим статистику на тестовых данных
    test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        test_labels = data[1].to(device)
        test_outputs = rnn_init(data[0].to(device))
        
        # подсчет ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
        # подсчет метрики на тесте
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_right += (test_labels == pred_test_labels).sum()
    
    test_loss_history.append(test_loss.item())
    print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_right / test_running_total:.3f}')
        
print('Training is finished!')

Epoch [1/5]. Step [355/355]. Loss: 0.549. Acc: 0.619. Test loss: 0.559. Test acc: 0.683
Epoch [2/5]. Step [355/355]. Loss: 0.545. Acc: 0.685. Test loss: 0.607. Test acc: 0.701
Epoch [3/5]. Step [355/355]. Loss: 0.553. Acc: 0.698. Test loss: 0.592. Test acc: 0.703
Epoch [4/5]. Step [355/355]. Loss: 0.577. Acc: 0.703. Test loss: 0.589. Test acc: 0.706
Epoch [5/5]. Step [355/355]. Loss: 0.588. Acc: 0.707. Test loss: 0.534. Test acc: 0.706
Training is finished!


# Какие проблемы у рекурентных сетей?

- затухают градиенты
- медленно, нужно всегда дойти до конца

<img src='https://drive.google.com/uc?export=view&id=17VtpKmVwlu0a7UjGbNfT0cjljfpsuKqU' width=800>

Вот у нас есть рекуррентная сеть. Она дает себе на вход значение на следующем шаге. Когда мы ее тренируем, мы как бы разматываем всю эту систему в одну большую сеть, которая прогоняет все элементы последовательности. И у всех слоев сетей в разные промежутки времени одни и те же веса. Соответсвенно, проходясь обратным распространением по этим сетям все это вместе складываем и применяем. Так происходит обучение.  

И от этого возникает проблема длинных зависимостей. Т.е. например, то что произошло в начале последовательности, может повлиять на то, что произойдет в конце этой последовательности. Для этого нужно, что бы сигнал во время тренировки протек по длинному пути всей последовательности. А у нас тут очень много матричных умножений на одну и ту же матрицу. Например, у нас 100 таких шагов. Что бы градиент прошел обратно, он будет сто раз умножен на одну и ту же матрицу. И это критично. Т.к. если, например, эта матрица какой-то один сигнал увеличивает чуть. То после того, как оно пройдет 100 раз, сеть этот сигнал сделает огромным. А у нас там tanh, который этот сигнал совсем убьет, т.к. сделает его очень большшим. 

Поэтому в такой простой формулировке, большие последовательности не получается тренировать. И на практике используют другие архитектуры, которые эту проблему решают.

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Hyperbolic_Tangent.svg/2560px-Hyperbolic_Tangent.svg.png' width=500>

# 2. LSTM. Long Short-Term Memory

[оригинальная статья. 1997](https://www.researchgate.net/publication/13853244_Long_Short-term_Memory)

Автор статьи [Юрген Шмидхубер](https://ru.wikipedia.org/wiki/%D0%A8%D0%BC%D0%B8%D0%B4%D1%85%D1%83%D0%B1%D0%B5%D1%80,_%D0%AE%D1%80%D0%B3%D0%B5%D0%BD). Первая реализации ее случилась только после несколько лет после ее публикации. 

[статья](http://colah.github.io/posts/2015-08-Understanding-LSTMs/), которая объясняет что там происходит

[перевод](https://alexsosn.github.io/ml/2015/11/17/LSTM.html) статьи выше

Вот как можно представить схему LSTM:

<img src='https://drive.google.com/uc?export=view&id=1h2F2sb-DgesuXRtLa4GbxH8QI_4swaE7' width=800>

Выглядит запутанно. Давайте распутывать.  

Основная идея.  

Теперь на каждом шаге мы передаем не один вектор, а два. Один из них (нижний) - это то самый h, который пойдет на вход следующему слою. А кроме этого, мы будем передавать, так называемое self-state. Назовем этот вектор - с. И вот h будет теперь проходить через много нелинейностей, умножаться на матрицы и будет испытывать все те же проблемы, которые мы обсуждали выше.  

А с будет как можно больше напрямую передаваться из прошлого состояния в следующее. Мы по нему будем очень аккуратно делать апдейты, что бы он очень плавно перетекал из одного состояния в другое.  

### 2.1 Как это выглядит подробнее:  

Внутри есть мнгого разных гейтов - некоторый вектор коэффициентов, которые соответсвуют той же размерности что и вход на них.  

#### **Forget gate**

Первый шаг в LSTM - это решить, от какой информации мы хотим избавиться. Это решение принимает слой с сигмоидой, который называется "forget gate layer." (гейт забывания). Он принимает во внимание $h_{t−1}$ и $x_t$, а на выходе даёт значение между 0 и 1 для каждого числа в состоянии ячейки $C_{t−1}$. 1 значит "полностью сохрани это", а 0 - "полностью забудь это".

Пример забывания - языковая модель пытается предсказать следующее слово базируясь на предыдущих. Здесь модель может запоминать род объекта, чтобы использовать правильное образование слов. Когда мы видим новый объект, то нужно забыть род предыдущего объекта. *(Животное не переходило дорогу, потому что оно устало)*

<img src='https://drive.google.com/uc?export=view&id=1oYGrHhH6y4_DtwRcKJDjzvYVChPEMblR'>

#### **Input gate**

Следующий шаг - решить, какую информацию мы должны хранить в состоянии ячейки. Шаг состоит из двух частей. Первая - слой сигмоиды, называемый "input gate layer" (входной гейт), который решает какие значения будут обновляться. Вторая - слой с тангенсом, который создает вектор значений $\tilde{C}_t$, которые будут добавляться к состоянию ячейки. Используем tanh, потому что нам хочется что бы эта добавка могла пойти и в + и в -. 


С примером языковой модели, мы бы хотели добавлять род нового объекта в состояние ячейки, чтобы заменить старый род, который мы забудем. *(Животное не переходило дорогу, потому что оно устало)*

<img src='https://drive.google.com/uc?export=view&id=1kb4hs0Y1cLb55sl-iRPsxMuNzLlu5qXb'>

#### **Update cell state**

Сейчас самое время, чтобы обновить старое состояние $C_{t−1}$ в новое состояние $C_t$. Предыдущие шаги уже решили, что делать, нужно только сделать это.

Умножаем старое состояние на $f_t$, тем самым забывая те вещи, которые хотели забыть, затем прибавляем $i_t∗\tilde{C_t}$. Это новое значение состояния ячейки, которое отмасштабировано в зависимоcти от того, насколько мы хотим обновить новое значение.

В языковой модели, это момент, где мы выкидываем информацию о роде старого объекта и добавляем новую информацию о роде нового объекта. *(Животное не переходило дорогу, потому что оно устало)*


<img src='https://drive.google.com/uc?export=view&id=1oKlTuYYRwdnvfMUQW0FjHRWgtHy0xl1M'>

#### **Output gate**

Наконец-то нам нужно решить, что мы отправим на выход. Выход будет базироваться на состоянии ячейки, но с небольшой фильтрацией. Во-первых, прогоним входной сигнал через сигмоиду, которая решает с какой силой дальше пропускать сигнал, во-вторых, прогоняем состояние ячейки через тангенс и умножаем это на сигмоиду, чтобы пропускать дальше только то, что мы решили пропустить.


Для языковой модели, которая видит только объект, здесь можем пропустить информацию, связанную с глаголом. Например, на выходе может быть полезно число множественной или единственное у объекта, чтобы знать в какую форму нужно поставить глагол. *(Животное не переходило дорогу, потому что оно устало)*

<img src='https://drive.google.com/uc?export=view&id=12SBxiBO-knE250rVlTxSYkmGWLjzFHL0'>

Вот так модуль LSTM выглядит в деталях. Зазубривать все это не обязательно. Основная идея зачем мы все это делаем - это бы мы на следующий шаг передавали два вектора. Один из них h в процессе своего формирования прошел через огромное количество нелинейностей, одних и тех же весов и т.д. и по нему градиент идет ни хорошо, ни плохо.  

Зато у этого С вычисления очень прямолинейные. Т.е. он умножен на какое-то число, которое еще и чаще всего 1, в нему была добавка. А функция + очень хороша для градиента. Вот почему:

Вот наш проход с. Вектор с умножается, складывается и идет дальше:


<img src='https://drive.google.com/uc?export=view&id=1IcyH_3Ab-KenQ8BgHeOrt44CcJ_1Wb_t' width=400>

А вот если все это сложить прошлое со следующим. То видим что мы организовали такой некий highway на котором градиент меньше всего затухает.

<img src='https://drive.google.com/uc?export=view&id=1C-EYALdJNO24YPxBhreZv2PitbQFQ73o' width=800>

Можно провести аналогию с тем, что происходит на сетях ResNet.

Визуализация прохода сигнала по LSTM:

<img src='https://drive.google.com/uc?export=view&id=1ezEAemVgiEW3POhGjEjIjFdre8I7pri-' width=600>

In [ ]:
from torch import nn


class LSTMFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, ht = self.lstm(x)
       
        if self.use_last:
            last_tensor = lstm_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(lstm_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        # print(out.shape)
        return torch.sigmoid(out)

In [ ]:
lstm_init = LSTMFixedLen(max_words, 128, 20, use_last=False)
optimizer = torch.optim.Adam(lstm_init.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [ ]:
print(lstm_init)
print("Parameters:", sum([param.nelement() for param in lstm_init.parameters()]))

LSTMFixedLen(
  (embeddings): Embedding(2000, 128, padding_idx=0)
  (lstm): LSTM(128, 20, num_layers=2, batch_first=True)
  (linear): Linear(in_features=20, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 271381


In [ ]:
lstm_init = lstm_init.to(device)
lstm_init.train()
th = 0.5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    lstm_init.train()
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        # обнуляем градиент
        optimizer.zero_grad()
        outputs = lstm_init(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
        
    # выводим статистику о процессе обучения
    lstm_init.eval()
    
    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
            f'Step [{i + 1}/{len(train_loader)}]. ' \
            f'Loss: {loss:.3f}. ' \
            f'Acc: {running_right / running_items:.3f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

    # выводим статистику на тестовых данных
    test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        test_labels = data[1].to(device)
        test_outputs = lstm_init(data[0].to(device))
        
        # подсчет ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
        # подсчет метрики на тесте
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_right += (test_labels == pred_test_labels).sum()
    
    test_loss_history.append(test_loss.item())
    print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_right / test_running_total:.3f}')
        
print('Training is finished!')

Epoch [1/5]. Step [355/355]. Loss: 0.562. Acc: 0.632. Test loss: 0.585. Test acc: 0.690
Epoch [2/5]. Step [355/355]. Loss: 0.590. Acc: 0.692. Test loss: 0.578. Test acc: 0.700
Epoch [3/5]. Step [355/355]. Loss: 0.514. Acc: 0.703. Test loss: 0.567. Test acc: 0.702
Epoch [4/5]. Step [355/355]. Loss: 0.595. Acc: 0.709. Test loss: 0.551. Test acc: 0.707
Epoch [5/5]. Step [355/355]. Loss: 0.537. Acc: 0.712. Test loss: 0.516. Test acc: 0.709
Training is finished!


# 3. GRU. Gated Recurrent Unit


Какие проблемы всё еще есть с LSTM:

- вычислительно сложно -> медленнее
- на очень длинных последовательностях все равно затухает градиент


Зачем платить больше - уберем некоторые врата (точнее совместим) -> ускоримся, уменьшим число параметров -> GRU

Но какого-то большого выигрыша от этого нет и наиболее часто применяются LSTM. 
И GRU:

<img src='https://drive.google.com/uc?export=view&id=10Q_1hrXcOlpe5WgIq-tuGBScDzaxH_Vl' width=600>

In [ ]:
from torch import nn


class GRUFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True, )
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        gru_out, ht = self.gru(x)
       
        if self.use_last:
            last_tensor = gru_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(gru_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        return torch.sigmoid(out)

In [ ]:
gru_init = GRUFixedLen(max_words, 128, 20, use_last=False)
optimizer = torch.optim.Adam(gru_init.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [ ]:
print(gru_init)
print("Parameters:", sum([param.nelement() for param in gru_init.parameters()]))

GRUFixedLen(
  (embeddings): Embedding(2000, 128, padding_idx=0)
  (gru): GRU(128, 20, num_layers=2, batch_first=True)
  (linear): Linear(in_features=20, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Parameters: 267541


In [ ]:
gru_init = gru_init.to(device)
gru_init.train()
th = 0.5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs): 
    gru_init.train() 
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        # обнуляем градиент
        optimizer.zero_grad()
        outputs = gru_init(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
        
    # выводим статистику о процессе обучения
    gru_init.eval()
    
    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
          f'Step [{i + 1}/{len(train_loader)}]. ' \
          f'Loss: {loss:.3f}. ' \
          f'Acc: {running_right / running_items:.3f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

    # выводим статистику на тестовых данных
    test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        test_labels = data[1].to(device)
        test_outputs = gru_init(data[0].to(device))
        
        # подсчет ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
        # подсчет метрики на тесте
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_right += (test_labels == pred_test_labels).sum()
    
    test_loss_history.append(test_loss.item())
    print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_right / test_running_total:.3f}')
            
print('Training is finished!')

Epoch [1/5]. Step [355/355]. Loss: 0.586. Acc: 0.632. Test loss: 0.617. Test acc: 0.689
Epoch [2/5]. Step [355/355]. Loss: 0.539. Acc: 0.695. Test loss: 0.546. Test acc: 0.703
Epoch [3/5]. Step [355/355]. Loss: 0.537. Acc: 0.703. Test loss: 0.499. Test acc: 0.704
Epoch [4/5]. Step [355/355]. Loss: 0.553. Acc: 0.709. Test loss: 0.490. Test acc: 0.705
Epoch [5/5]. Step [355/355]. Loss: 0.525. Acc: 0.712. Test loss: 0.567. Test acc: 0.707
Training is finished!


# 4. Интересные концепты

## 4.1 Attention

Примеры из статьи [Google's Neural Machine Translation System: Bridging the Gap between Human and Machine Translation](https://arxiv.org/abs/1609.08144).

<img src='https://drive.google.com/uc?export=view&id=1uOHQX8AnPGKY6HEDOTRJzVxCiKAHLoJy' width=450>

Нужно сжать всю информацию о предыдущих словах в один блок, который представляет собой внутренний слой - переходное состояние из кодировщика в декодировщика, что очень сложная задача, которая приводит к недообучению.

И одно из возможных решений - это настакать слои LSTM, один слой LSTM создает вход для другого слоя LSTM:

<img src='https://drive.google.com/uc?export=view&id=1yrBxb5DXZA-LNwNWQdmuF4TNNXqTfNpJ' width=450>

Но эту вещь очень тяжело обучать, если добавлять всё больше LSTM слоем, то снова встречаемся с проблемой затухающих градиентов, плюсом, всё равно нужно сжимать всю информацию в последние блоки - это наш bottleneck (узкое место, горлышко бутылки) между кодировщик и декодировщиком.


И решение этой проблемы - Attention слой. Вместо того, чтобы сжимать всю информацию из всех временных ячеек в одно скрытое состояние, можно дать доступ декодировщику ко всей истории. Но получаем очень слишком много информации, которую нужно учитывать, поэтому мы будем обращать свое внимание только на подвыборку этих ячеек.

Будем учитывать, какая часть предложения на английском языке важна для предсказания слова на французском языке. Подсчет такого распределения достигается получением оценки релевантности каждого слова в предложении для получения нового слова.

<img src='https://drive.google.com/uc?export=view&id=1GAVWyShMSXELaz1f-_e4c9rAeDoJ3QE_' width=450>


**Как использовать эти оценки релевантности?**

Подсчитываются оценки для каждого скрытого состояния и образуют собой взвешанную сумму внимания. А потом эта сумма передается, как еще один вход в декодировщик.

<img src='https://drive.google.com/uc?export=view&id=1a72Ake6Ai2SdkZ9cQpqmq6-ebjblcQ34' width=450>

Можно [провизуализировать](https://distill.pub/2016/augmented-rnns/#attentional-interfaces), какое внимание уделяется каждому слову из предложения:

<img src='https://drive.google.com/uc?export=view&id=1OJ4vgTHPxGMzcR8Ob8An--FFUlnZabQ3' width=500>

А ещё слои внимания можно добавлять и не только к текстам, но и к картинкам.

<img src='https://drive.google.com/uc?export=view&id=1i48NmZMXUneeUwW339tSgvrGU_VvLh9H'>



## 4.2 Bidirectional RNN

Из класса RNN есть еще интересный вариант [Bidirectional RNN](https://maxwell.ict.griffith.edu.au/spl/publications/papers/ieeesp97_schuster.pdf). Они позволяют видеть не только прошлое состояние, но и будущее.


<img src='https://drive.google.com/uc?export=view&id=1kCGsUWhUjIoIAquvE7FjaAbYjAQdsyW7' width=550>


Общая идея: на каждом шаге есть и прямой проход и обратный. И векторы которые несут информацию из прошлого и будущего конкатенируются вместе и идут на выход с сети.

# Домашнее задание

1. Попробуйте обучить нейронную сеть GRU/LSTM для предсказания сентимента сообщений с твитера на примере https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech

2. Опишите, какой результат вы получили? Что помогло вам улучшить ее точность?

У кого нет возможности работать через каггл (нет верификации), то можете данные взять по ссылке: https://disk.yandex.ru/d/LV1cYS1orMyRWA